In [4]:
import sqlite3
import json

In [5]:
from datetime import datetime

In [6]:
timeframe = '2005-12'

In [7]:
connection = sqlite3.connect('{}.db'.format(timeframe))

In [8]:
c = connection.cursor()

In [9]:
c.execute("""create table if not exists parent_reply 
(parent_id text primary key, comment_id text unique, parent text,
comment text, subreddit text, unix int, score int)""")

In [42]:
c.execute("""delete from parent_reply""")

In [43]:
connection.commit()

In [46]:
for row in c.execute("""select * from parent_reply"""):
    print row

In [27]:
def format_data(data):
    data = data.replace("\n", " newlinechar ").replace("\r", " newlinechar ").replace('"', "'")
    return data

In [26]:
def find_parent(parent_id):
    try: 
        sql = "select comment from parent_reply where comment_id = '{}' limit 1".format(parent_id)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        return False 

In [29]:
def find_existing_score(parent_id):
    try: 
        sql = "select score from parent_reply where parent_id = '{}' limit 1".format(parent_id)
        c.execute(sql)
        result = c.fetchone()
        if result != None:
            return result[0]
        else: return False
    except Exception as e:
        return False 

In [32]:
def acceptable(data):
    if len(data.split(' ')) > 50 or len(data) < 1:
        return False
    elif len(data) > 1000:
        return False
    elif data == '[deleted]' or data == '[removed]':
        return False
    else: return True

In [37]:
def sql_insert_replace_comment(comment_id, parent_id, 
                         parent_data, body, subreddit, created_utc, score):
    try:
        sql = """update parent_reply set parent_id=?, comment_id=?,
        parent_data=?, body=? subreddit=?, unix=?, score=? where
        parent_id=?;""".format(parent_id, comment_id, parent_data, body,
                        subreddit, created_utc, score, parent_id)
        c.execute(sql)
        connection.commit()
        
    except Exception as e:
        print('s-UPDATE insertion', str(e))

In [38]:
def sql_insert_has_parent(comment_id, parent_id, 
                         parent_data, body, subreddit, created_utc, score):
    try:
        sql = """insert into parent_reply values(parent_id, comment_id,
        parent_data, body, subreddit, created_utc, score)""" 
        c.execute(sql)
        connection.commit()
        
    except Exception as e:
        print('s-PARENT insertion', str(e))

In [39]:
def sql_insert_no_parent(comment_id, parent_id, 
                         body, subreddit, created_utc, score):
    try:
        sql = """insert into parent_reply      
        (parent_id, comment_id, comment, subreddit, unix, score)
        values(parent_id, comment_id, body, subreddit, created_utc, 
        score)""" 
        c.execute(sql)
        connection.commit()
        
    except Exception as e:
        print('s-NO PARENT insertion', str(e))

In [47]:
row_counter = 0
paired_rows = 0

with open("RC_2005-12") as f:
    for row in f:
        row_counter+=1
        row = json.loads(row)
        parent_id = row['parent_id']
        body =  format_data(row['body'])
        created_utc = row['created_utc']
        score = row['score']
        subreddit = row['subreddit']
        comment_id = row['link_id']
        print_data = find_parent(parent_id)
        
        if score >= 2:
            if(acceptable(body)):
                existing_comment_score = find_existing_score(parent_id)
                if existing_comment_score:
                    if score > existing_comment_score:
                        sql_insert_replace_comment(comment_id, parent_id, 
                         parent_data, body, subreddit, created_utc, score)
                        
                    else: 
                        if parent_data:
                            sql_insert_has_parent(comment_id, parent_id, 
                         parent_data, body, subreddit, created_utc, score)
                            paired_rows+=1
                        else: 
                            sql_insert_no_parent(comment_id, parent_id, 
                            body, subreddit, created_utc, score)
        
        print("Total rows read: {}, Total pair rows reads: {}".format(row_counter, paired_rows))
        

Total rows read: 1, Total pair rows reads: 0
Total rows read: 2, Total pair rows reads: 0
Total rows read: 3, Total pair rows reads: 0
Total rows read: 4, Total pair rows reads: 0
Total rows read: 5, Total pair rows reads: 0
Total rows read: 6, Total pair rows reads: 0
Total rows read: 7, Total pair rows reads: 0
Total rows read: 8, Total pair rows reads: 0
Total rows read: 9, Total pair rows reads: 0
Total rows read: 10, Total pair rows reads: 0
Total rows read: 11, Total pair rows reads: 0
Total rows read: 12, Total pair rows reads: 0
Total rows read: 13, Total pair rows reads: 0
Total rows read: 14, Total pair rows reads: 0
Total rows read: 15, Total pair rows reads: 0
Total rows read: 16, Total pair rows reads: 0
Total rows read: 17, Total pair rows reads: 0
Total rows read: 18, Total pair rows reads: 0
Total rows read: 19, Total pair rows reads: 0
Total rows read: 20, Total pair rows reads: 0
Total rows read: 21, Total pair rows reads: 0
Total rows read: 22, Total pair rows reads:

In [45]:
#c.execute("""insert into parent_reply values('ticiana', 'ticiana',
#'ticiana', 'ticiana', 'ticiana', 1, 0)""")